In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from skimage.morphology import remove_small_holes, area_opening, skeletonize
import pandas as pd
from matplotlib import rcParams

In [2]:
COLOR = 'black'
plt.style.use("default")
rcParams["font.family"] = "serif"
rcParams["mathtext.fontset"] = "cm"
rcParams['figure.figsize'] = (28, 18)
rcParams['figure.titlesize'] = 20
rcParams['figure.dpi'] = 50
rcParams['axes.grid'] = True
rcParams['axes.facecolor'] = 'lightgray'
rcParams['legend.fontsize'] = 25
rcParams['axes.titlesize'] = 60
rcParams['axes.labelsize'] = 50
rcParams['ytick.labelsize'] = 20
rcParams['xtick.labelsize'] = 20
rcParams['text.color'] = COLOR
rcParams['text.color'] = COLOR
rcParams['axes.labelcolor'] = COLOR
rcParams['xtick.color'] = COLOR
rcParams['ytick.color'] = COLOR
rcParams['grid.color'] = COLOR
rcParams['text.latex.preamble'] = "\n".join([
    r'\usepackage[version=3]{mhchem}'
])

In [3]:
THRESHOLD = 120
MAX_PIXEL_VALUE = 255

### funciones

In [65]:
def cortar(frame, limites, PRUEBA=False, nombre='CORTE'):
    # lee en gris
    min_x, max_x, min_y, max_y = limites
    corte = frame[min_y:max_y, min_x:max_x] # resolver lo del corte binarizadp
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', corte)
    return corte

def gris(frame, PRUEBA=False, nombre='GRAYSCALE'):
    im_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', im_gray)
    return im_gray

def binarize(frame, th=THRESHOLD, PRUEBA=False, nombre='BINARY'):
    # im_binary = cv.adaptiveThreshold(frame, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    frame = gris(frame)
    _, im_binary = cv.threshold(frame, th, MAX_PIXEL_VALUE, cv.THRESH_BINARY)
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', im_binary)
    return im_binary

def guillotina(frame, PRUEBA=False, nombre='DECAPITADO'):
    y, x = np.where(frame<=130)
    frame[y, x] = np.max(frame)
    frame -= np.min(frame)
    frame = (frame/np.max(frame))*255
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', frame)
    return frame

def binarize_TAIL(frame, PRUEBA=False, nombre='BINARY_TAIL'):
    _, im_binary = cv.threshold(frame, THRESHOLD, MAX_PIXEL_VALUE, cv.THRESH_BINARY)
    arr = im_binary > 0
    im_binary = remove_small_holes(arr)*255
    im_binary = area_opening(im_binary, 200)
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', im_binary)
    return im_binary

def skeleton(frame, PRUEBA=False, nombre='ESQUELETO'):
    arr = frame < 255
    sk = skeletonize(arr)*255
    y, x = np.where(sk!=0)
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', sk)
    return x, y

def track_TAIL(vs, limites, SHOW=False):
        # Crea las listas vacias que van a ser nuestras mediciones
        frame_count = vs.get(7)
        tiempo = [f for f in range(1, int(frame_count))]
        while(vs.isOpened()):
            ret, frame = vs.read()
            if ret:
                frame = cortar(frame, limites)
                im = frame[:, -2:-1]
                im = cv.bitwise_not(binarize(im, th=170))
                if np.sum(im) == 0:
                    frame_bn = binarize_TAIL(guillotina(gris(frame)))
                    try:
                        x, y = skeleton(frame_bn)
                        frame[y, x] = [0, 0, 255]
                        cv.imshow('frame', frame)
                    except:
                        break
                    if cv.waitKey(1) & 0xFF == ord('q'):
                        break
                    elif
            else:
                break
        cv.destroyAllWindows()
        # video.release()
                
        return np.array(tiempo)

In [6]:
lim = [319, 950, 250, 505]

In [64]:
im = cv.imread('Capturas\\Camo Snapshot 2024-05-17 12-04-24 - 10Hz.jpg')
im = cortar(im, limites=lim)
im = im[:, -2:-1]
im = binarize(im, th=170, PRUEBA=True, nombre='algo')
# print(im)
im = cv.bitwise_not(im)
print(np.sum(im)==0)
im_gray = gris(im_cut)
im_decap = guillotina(im_gray, PRUEBA=True)
im_binary = binarize_TAIL(im_decap, PRUEBA=True)
im_sk = skeleton(im_binary, PRUEBA=True)

True


In [14]:
video = cv.VideoCapture('ECO20-45mm-PECERA1\ECO20-45mm-2,04Vpp-3Hz.mp4')
t = track_TAIL(video, lim, SHOW=True)

<>:1: SyntaxWarning: invalid escape sequence '\E'
<>:1: SyntaxWarning: invalid escape sequence '\E'
C:\Users\pauli\AppData\Local\Temp\ipykernel_30088\2234935015.py:1: SyntaxWarning: invalid escape sequence '\E'
  video = cv.VideoCapture('ECO20-45mm-PECERA1\ECO20-45mm-2,04Vpp-3Hz.mp4')
